In [2]:
import numpy as np
import pandas as pd

# 1. Load datasets

In [2]:
df = pd.read_pickle('datasets/refined/recipes_refined_merged.pkl')

In [3]:
list(df[df["id"] == 221185]["description"])

['My friend gave me some trout a couple months ago and I used this recipe out of "Louisiana Cookin\'" and it was divine!']

In [4]:
df[df["name"].str.contains("&amp;")]

,id,name,author_name,submitted,image_url,minutes,description,category,tags,search_terms,steps,ingredients,ingredients_raw_str,serving_size,servings,calories,rating,review_count
6,232076,Retro Chicken &amp; Chips Casserole,Kizzikate,2007-06-03,"""https://img.sndimg.com/food/image/upload/w_55...",25,"From Cooking Light Magazine, 11/05. This is co...",One Dish Meal,"['30-minutes-or-less', 'time-to-make', 'course...","['casserole', 'dinner', 'chicken']","['In large bowl, combine chicken, green onion,...","['chicken breasts', 'green onion', 'red bell p...","[""4 cups roasted chopped chicken breasts"",...",1 (85 g),6,515.5,4.5,4
25,232105,Simple Garlic &amp; Pepper Rack of Lamb,Jen T,2007-06-04,"""https://img.sndimg.com/food/image/upload/w_55...",35,This is an ideal way of cooking racks of lamb ...,Lamb/Sheep,"['60-minutes-or-less', 'time-to-make', 'course...",['dinner'],"[""Preheat oven to 350'F."", 'Heat oil in pan an...","['racks of lamb', 'oil', 'garlic', 'peppercorn...","[""2 racks of lamb (frenched \u0026 fatty...",1 (7 g),2,72.6,5.0,3
39,127155,Crab &amp; Fresh Basil Stuffed Mushrooms,NcMysteryShopper,2005-06-22,"""https://img.sndimg.com/food/image/upload/w_55...",35,A wonderful tasting and attractive hors d’oeuv...,Crab,"['60-minutes-or-less', 'time-to-make', 'course...","['appetizer', 'low-calorie', 'dinner', 'low-ca...","['Combine cheese, breadcrumbs, salt, green oni...","['monterey jack cheese', 'plain breadcrumbs', ...","[""3 cups monterey jack cheese, shredded ...",1 (185 g),8,272.2,5.0,7
49,204805,Spicy Hot &amp; Sour Garlic Saute',Sister Earth,2007-01-11,NaN,40,This is a fun & easy way to get an exotic spic...,Curries,"['curries', '60-minutes-or-less', 'time-to-mak...",['dinner'],"[""Have the meat and/or veggies that you are go...","['meat', 'garlic cloves', 'fresh gingerroot', ...","[""14 -18 ounces of stir fry meat (or a com...",1 (40 g),4,147.1,NaN,0
110,241335,Chicken Thighs With Lemon &amp; Garlic,Boonyas mom,2007-07-19,"""https://img.sndimg.com/food/image/upload/w_55...",1,This is a Mediterranean/Greek recipe. It's ver...,Chicken Thigh & Leg,"['60-minutes-or-less', 'time-to-make', 'course...","['low-sodium', 'greek', 'low-carb', 'dinner', ...","['Preheat oven to 350°.', 'Bring stock, garlic...","['chicken stock', 'garlic cloves', 'butter', '...","[""2 1/2 cups chicken stock"",""20 garl...",1 (429 g),4,545.2,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494691,43025,Asparagus &amp; Cheese Stuffed Turkey or Chick...,spatchcock,2002-10-14,NaN,80,This is a great dish for company.,Chicken,"['weeknight', 'time-to-make', 'course', 'main-...","['dinner', 'chicken']","['Cut cheese into approximately 1/2"" square cu...","['skinless boneless turkey breasts', 'swiss ch...","[""2 -3 lbs skinless boneless turkey breas...",1 (140 g),8,204.4,NaN,0
494853,207778,Wine &amp; Blue Cheese Hamburgers (Oamc),Oolala,2007-01-26,NaN,20,Called Different Hamburgers by Gloria Diggs wh...,Lunch/Snacks,"['30-minutes-or-less', 'time-to-make', 'course...","['dinner', 'lunch']","['Combine all ingredients except the beef, in ...","['ground beef', 'salt', 'pepper', 'blue cheese...","[""1 lb ground beef"",""1/2 teaspoon sal...",1 (115 g),4,386.3,3.0,2
494859,141156,Light &amp; Creamy Layered Lemon Cheesecake,hannahactually,2005-10-12,"""https://img.sndimg.com/food/image/upload/w_55...",95,"I am not a big cheesecake fan, but this is abs...",Cheesecake,"['time-to-make', 'course', 'preparation', 'occ...","['dessert', 'vegetarian']","['To make bottom layer:.', 'Mix flour, margari...","['flour', 'margarine', 'nuts', 'cream cheese',...","["""",""1 cup flour"",""1/2 cup margarine""...",1 (2061 g),1,5575.2,5.0,1
494875,499832,Peanut Butter &amp; Jelly Waffles,ElizabethKnicely,2013-05-04,NaN,1,Don't count out the grownups when it comes to ...,Breakfast,"['weeknight', '60-minutes-or-less', 'time-to-m...",['breakfast'],"['In a large bowl, combine flour, sugar, bakin...","['all-purpose flour', 'sugar', 'baking powder'...","[

---

# 2. Analysing

In [5]:
list(df[df["id"] == 107786]["image_url"])

['"https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/10/77/86/mr5ViaW4TBmhRU3uCtLB_0S9A8585.jpg"']

### Counting tag occurrence

---

# 3. Charactersitic update

### "&;amp;" to "&"

In [6]:
df["name"] = df["name"].str.replace("&amp;", "&")

In [5]:
target_columns = ["tags", "search_terms", "steps", "ingredients", "ingredients_raw_str"]

### Remove qoutes in desc and name

In [8]:
for col in ["name", "description", "steps"]:
    df[col] = df[col].str.replace('"',"").str.replace("'","")

In [9]:
list(df[df["id"] == 20812]["steps"])

['{Mix graham cracker crumbs; sugar and margarine in a 13 x 9 cake pan.; Press evenly onto bottom of pan.; Arrange banana slices on crust.; Beat cream cheese in a large bowl with a wire whisk or electric mixer until smooth.; Gradually beat in milk.; Add pudding mixes.; Beat until well blended.; Spread evenly over banana slices.; Spoon pineapple evenly over pudding mixture.; Spread Whipped topping over pineapple.; Refrigerate for 3 hours before serving.}']

### Change quote marks for Postgres 

In [10]:
for col in ["tags", "search_terms", "steps", "ingredients"]:
    df[col] = df[col].str.replace("'",'"').str.replace('", "','","')

### Separator malfunction fix

In [11]:
for col in ["steps", "ingredients_raw_str"]:
    df[col] = df[col].str.replace('","','|').str.replace(',',';').str.replace('|','","')#.str.replace('"",','').str.replace(',""','')

/tmp/ipykernel_66/221723067.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('","','|').str.replace(',',';').str.replace('|','","')#.str.replace('"",','').str.replace(',""','')


In [12]:
for col in target_columns:
    df[col] = df[col].str.replace('"",','').str.replace(',""','')

### Section save 

In [13]:
df.to_pickle('datasets/refined/recipes_temp1.pkl')

In [3]:
df = pd.read_pickle('datasets/refined/recipes_temp1.pkl')

### Inline bounders replacement

### Change brackets 

In [6]:
for col in target_columns:
    df[col] = "{" + df[col].str[1:-1].str.replace("{","(").str.replace("}",")") + "}"

/tmp/ipykernel_86/2558694278.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = "{" + df[col].str[1:-1].str.replace("{","(").str.replace("}",")") + "}"


### Apostrophe fix
(Only if needed)

# 4. Save

In [7]:
df.to_csv('datasets/refined/refined_merged_char_recipes.csv', sep=';', index=False)

In [8]:
df.to_pickle('datasets/refined/refined_merged_char_recipes.pkl')